# Libraries and Global Variables

In [18]:
import pandas as pd
import os
import numpy as np
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

import arcpy

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [19]:
# web app folder
folderWebAppDataFolder = r"..\widgets\Resiliency\data"

In [20]:
working_directory = os.getcwd()
data_folder         = os.path.join(working_directory, "data"        )
intermediate_folder = os.path.join(working_directory, "intermediate")
results_folder      = os.path.join(working_directory, "results"     )

print(data_folder)
print(intermediate_folder)
print(results_folder)

e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\data
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\intermediate
e:\WebAppBuilderForArcGIS_v2.20\server\apps\15\_dataprep\results


In [21]:
processGdb = os.path.join(intermediate_folder,'process.gdb')
processGdb

'e:\\WebAppBuilderForArcGIS_v2.20\\server\\apps\\15\\_dataprep\\intermediate\\process.gdb'

In [22]:
def delIfExists(obj):
    try:
        if arcpy.Exists(obj):
            arcpy.Delete_management(obj)
            #print ("DELTED: " + obj)
        #else:
            #print ("DOESN'T EXIST: " + obj)
    except:
        #print ("PASSED: " + obj)
        pass

In [23]:
dfProjects = pd.DataFrame([
    ['Roadway','points',r'RTP_2023_Roadway.gdb\RTP_2023_Roadway_points'],
    ['Roadway','lines' ,r'RTP_2023_Roadway.gdb\RTP_2023_Roadway_lines' ],
    ['Transit','points',r'RTP_2023_Transit.gdb\RTP_2023_Transit_points'],
    ['Transit','lines' ,r'RTP_2023_Transit.gdb\RTP_2023_Transit_lines' ],
    ['AT'     ,'points',r'RTP_2023_AT.gdb\RTP_2023_AT_points'          ],
    ['AT'     ,'lines' ,r'RTP_2023_AT.gdb\RTP_2023_AT_lines'           ],
],columns=('prjGroup','prjType','prjFC'))


# Download Data

In [31]:
dfCategories = pd.DataFrame([
  ["AF", "Agricultural and Farmland"            ],
  ["CH", "Cultural and Historic Resources"      ],
  ["EJ", "Environmental Justice Consideration"  ],
  ["FP", "Floodplains"                          ],
  ["GH", "Geological Hazards"                   ],
  ["HW", "Habitat and Wildlife"                 ],
  ["HM", "Hazardous Materials & Contamination"  ],
  ["HY", "Hydrological"                         ],
  ["OS", "Open Space, Parks, and Recreation"    ],
# ["LU", "Land Use"                             ], # DON'T USE FOR NOW
  ["SS", "Steep Slopes"                         ]
], columns=['CategoryCode','CategoryName'])
display(dfCategories)

# GOOGLE SHEEET
#https://docs.google.com/spreadsheets/d/1cpS4wcMXwt9Yd7qZisvJfZ8GcRLhFpcTRJS_l7CBBY0/edit#gid=0

# quick view layers: https://www.arcgis.com/apps/mapviewer/index.html?layers=74a6478eae424c17b96d1677ee1a4654

dfOpenDataLayers = pd.DataFrame([  
  [0, "OS", "LOWN", "State and Federal Lands"              , "State and Federal Lands"              , "929fe39efa1d43ac8b151c2898ad3efe", "OWNER!='Private'"       , 1.5],
  [0, "AF", "AGFL", "Agricultural and Farmland"            , "Agricultural and Farmland"            , "875527c1e41e40d8b52340f18e89fb52", ""                       , 1.5],
  [0, "HW", "ELKS", "Elk"                                  , "Elk Habitat"                          , "224dbcbdca174d3e8648a8b5693fd130", "VALUE=='crucial'"       , 1.5],
  [0, "HW", "DEER", "Mule Deer"                            , "Mule Deer Habitat"                    , "a4ea235cef824862b560c693b4f7959b", "VALUE=='crucial'"       , 1.5],
 #[0, "HW", "PRNG", "Pronghorn"                            , "Pronghorn Habitat"                    , "1cd6e0315a19473d925427f7cc1d02c1", "VALUE=='crucial'"       , 1.5], # no near WFRC
 #[0, "HW", "TURK", "Turkey"                               , ""                                     , "9654e665cccd4884adb6a416df4d3444", ""                       , 1.5], # no turkeys in crucial
  [0, "HW", "MOOS", "Moose"                                , "Moose Habitat"                        , "e66c061ebd4346e3b16ed8786f5d5d78", "VALUE=='crucial'"       , 1.5],
  [0, "HW", "BEAR", "Black Bear"                           , "Black Bear Habitat"                   , "6440f19a05c149b783baf1705e5f2a51", "VALUE=='crucial'"       , 1.5],
 #[0, "HW", "HARE", "Snowshoe Hare"                        , "Snowshoe Hare Habitat"                , "d151c91e95c74b329814319f3c20b145", "VALUE=='crucial'"       , 1.5], # none near WFRC
  [0, "HW", "GOAT", "Mountain Goat"                        , "Mountain Goat Habitat"                , "b6b369f865d243399bfc80908426d31b", "VALUE=='crucial'"       , 1.5],
  [0, "HW", "DGRS", "Dusky Grouse"                         , "Dusky Grouse Habitat"                 , "10382c7f5f384fc692d094fcb65bf251", "VALUE=='crucial'"       , 1.5],
 #[0, "HW", "RGRS", "Ruffed Grouse"                        , ""                                     , "a3d3fe1f13324882aa4b7f67253b1c06", ""                       , 1.5], # no crucial near WF
 #[0, "HW", "HPAR", "Hungarian Partridge"                  , ""                                     , "7a7c3a873f0148ab8a188f743b7acff2", ""                       , 1.5], # no crucial
  [0, "HW", "CHKR", "Chukar"                               , "Chukar Habitat"                       , "97c6f7c74f704f84b687eb0746cc07ec", "VALUE=='crucial'"       , 1.5],
  [0, "HW", "BISN", "Bison"                                , "Bison Habitat"                        , "9bed043eb87641db88424d1080d03045", "VALUE=='crucial'"       , 1.5],
 #[0, "HW", "BGHR", "Rocky Mountain Bighorn Sheep"         , ""                                     , "82b1e7ce1ec74920bcd40cd7e729c9b3", ""                       , 1.5], # none near WFRC
 #[0, "HW", "RNPH", "Ringed-Neck Pheasant"                 , ""                                     , "56acb32933f4463bb81cebb834595905", ""                       , 1.5], # none near WF
  [0, "HW", "CSTG", "Columbian Sharp-Tailed Grouse"        , "Columbian Sharp-Tailed Grouse Habitat", "c8e7ae2533d84a9a98268357dafa15da", "VALUE=='crucial'"       , 1.5],
  [0, "HW", "CAQU", "California Quail"                     , "California Quail Habitat"             , "d5dae6ecfc8d490b87f1f67fd4d16fd9", "VALUE=='crucial'"       , 1.5],
  [0, "HW", "CABS", "California Bighorn Sheep"             , "California Bighorn Sheep Habitat"     , "4c11e02a83f148dcafacbe203efc7432", ""                       , 1.5], # include all since value field looks mis-coded
  [0, "HW", "BTPG", "Band-Tailed Pigeon"                   , "Band-Tailed Pigeon Habitat"           , "524a4c4b704244d4979b4d2b56f1d937", "VALUE=='crucial'"       , 1.5],
 #[0, "HW", "GMQU", "Gambel's Quail"                       , ""                                     , "95a538966055498e956817bad0a5835c", ""                       , 1.5], # None near WF
 #[0, "HW", "WTPT", "White-Tailed Ptarmigan"               , ""                                     , "170dbdc20fcc4a138022738167c4a765", ""                       , 1.5], # None near WF
 #[0, "HW", "WWDV", "White-Winged Dove"                    , ""                                     , "a2d03a434d6b4b189784434014fe0eef", ""                       , 1.5], # None near WF
 #[0, "HW", "UDBH", "Utah Desert Bighorn Sheep"            , ""                                     , "8c19124fbce245b5942c9e48c0568fab", ""                       , 1.5], # None near WF
 #[0, "HW", "WTJR", "White-Tailed Jackrabbit"              , ""                                     , "1b02a508fa334d33ba18c0156d027751", ""                       , 1.5], # None near WF
  [0, "HM", "BFWT", "Brownfields Targeted for Cleanup"     , "Brownfields Targeted for Cleanup"     , "630351c0db444fe7933a08eab40e573e", ""                       , 1.5],
  [0, "HM", "BFNT", "Brownfields Not Targeted for Cleanup" , "Brownfields Not Targeted for Cleanup" , "5fd65304423d40b38b4d93e50698db70", ""                       , 1.5],
  [0, "HM", "HZWO", "Hazardous Waste & Used Oil Facilities", "Hazardous Waste & Used Oil Facilities", "8e9e6310622043bca68ffc2298959126", ""                       , 1.5],
 #[0, "HM", "LLWD", "Low-Level Waste & Uranium Mills"      , "Low-Level Waste & Uranium Mills"      , "2c81695cbc644afdba7687e0340b5763", ""                       , 1.5], # DO NOT USE
  [0, "HM", "SWLN", "Solid Waste Landfills"                , "Solid Waste Landfills"                , "1ce0d598a0e440558c63535661dc2399", ""                       , 1.5],
  [0, "HM", "FUDS", "Formerly Used Defense Sites"          , "Formerly Used Defense Sites"          , "dbcdc13c51614d109102f1251fe9530b", ""                       , 1.5],
  [0, "HM", "MMRP", "Military Munitions Response Program"  , "Military Munitions Response Program"  , "c77c53794aeb4c448e1307b31c581909", ""                       , 1.5],
 #[0, "HM", "EWAS", "Enforceable Written Assurances"       , "Enforceable Written Assurances"       , "7299aa0fdc80426ca0e42c42f59e1518", ""                       , 1.5],
  [0, "HM", "NPRL", "National Priorities List"             , "National Priorities List"             , "f9c01f05050c40d7b86dade7dd877cf9", ""                       , 1.5],
  [0, "HM", "SFST", "Superfunded Sites"                    , "Superfunded Sites"                    , "929a2e7a8d5347b8b3248a450cc42e33", ""                       , 1.5],
  [0, "HM", "ICBZ", "Institutional Control Buffer Zones"   , "Institutional Control Buffer Zones"   , "d8ea2ca541724277b01726d98197da32", ""                       , 1.5],
 #[0, "HM", "UTRI", "Utah Toxic Release Inventory Tier II" , "Utah Toxic Release Inventory Tier II" , "2f725ff758e24dc496b17f87b7301ed8", ""                       , 1.5],
  [0, "HM", "TXRI", "Toxic Release Inventory"              , "Toxic Release Inventory"              , "132ac94ba71d4e9b9bcffa57b07feb54", ""                       , 1.5],
  [0, "HM", "VCUP", "Voluntary Cleanup Program"            , "Voluntary Cleanup Program"            , "e0316bc7899d4e4896e88fe3a46aafa8", ""                       , 1.5],
 #[0, "HM", "PSTF", "Petroleum Storage Tank Facilities"    , "Petroleum Storage Tank Facilities"    , "4d31fef4543f449ebe04a84909a11023", ""                       , 1.5],
  [0, "CH", "UHDS", "Utah Historic Districts"              , "Utah Historic Districts"              , "eecae7fdf65f47aeae12ba817aa0d66b", ""                       , 1.5],
  [0, "EJ", "EQFA", "Equity Focus Areas"                   , "Equity Focus Areas"                   , "2ed6d9b418d942b79323e6be7209d63c", ""                       , 1.5],
  [0, "OS", "LCPK", "Local Parks"                          , "Local Parks"                          , "818536733987486b8082e4c5dec0a1dd", ""                       , 1.5],
  [0, "OS", "PAPR", "Public Access Properties (WMA)"       , "Public Access Properties (WMA)"       , "56aadb826ef3442db805553caf8126c0", ""                       , 1.5],
  [0, "FP", "FLDP", "Floodplains"                          , "Floodplains"                          , "9100130e33034813833a938069b7bf79", ""                       , 1.5],
  [0, "GH", "QUFL", "Quaternary Faults"                    , "Quaternary Faults"                    , "9c85978d0fb54570bc60bec467e2fa7f", ""                       , 1.5],
  [0, "SS", "STSL", "Steep Slopes"                         , "Steep Slopes"                         , "d42f60e142dd4366a1c24aae4993f8e3", ""                       , 1.5],
  [0, "HY", "RIPA", "Riparians"                            , "Riparians"                            , "74a6478eae424c17b96d1677ee1a4654", ""                       , 1.5],
  [0, "HY", "WETL", "Wetlands"                             , "Wetlands"                             , "dbc6a99d04944dab9f23bb6286843a1d", ""                       , 1.5]
],columns=['Download','CategoryCode','LayerCode','ListName','LayerName','OpenDataKey','QueryFilter','SearchBuffer'])
display(dfOpenDataLayers)

# example: https://opendata.arcgis.com/datasets/a37e501176dd400c9c7498c29e766ba2_0.gdb

#item  = gis.content.get("630351c0db444fe7933a08eab40e573e")
#layer = item.layers[0]

#dfDownloadedLayers = pd.DataFrame([
#  [],
#  [],
#],columns=['LayerName','Filename'])

,CategoryCode,CategoryName
0,AF,Agricultural and Farmland
1,CH,Cultural and Historic Resources
2,EJ,Environmental Justice Consideration
3,FP,Floodplains
4,GH,Geological Hazards
5,HW,Habitat and Wildlife
6,HM,Hazardous Materials & Contamination
7,HY,Hydrological
8,OS,"Open Space, Parks, and Recreation"
9,SS,Steep Slopes


,Download,CategoryCode,LayerCode,ListName,LayerName,OpenDataKey,QueryFilter,SearchBuffer
0,0,OS,LOWN,State and Federal Lands,State and Federal Lands,929fe39efa1d43ac8b151c2898ad3efe,OWNER!='Private',1.5
1,0,AF,AGFL,Agricultural and Farmland,Agricultural and Farmland,875527c1e41e40d8b52340f18e89fb52,,1.5
2,0,HW,ELKS,Elk,Elk Habitat,224dbcbdca174d3e8648a8b5693fd130,VALUE=='crucial',1.5
3,0,HW,DEER,Mule Deer,Mule Deer Habitat,a4ea235cef824862b560c693b4f7959b,VALUE=='crucial',1.5
4,0,HW,MOOS,Moose,Moose Habitat,e66c061ebd4346e3b16ed8786f5d5d78,VALUE=='crucial',1.5
5,0,HW,BEAR,Black Bear,Black Bear Habitat,6440f19a05c149b783baf1705e5f2a51,VALUE=='crucial',1.5
6,0,HW,GOAT,Mountain Goat,Mountain Goat Habitat,b6b369f865d243399bfc80908426d31b,VALUE=='crucial',1.5
7,0,HW,DGRS,Dusky Grouse,Dusky Grouse Habitat,10382c7f5f384fc692d094fcb65bf251,VALUE=='crucial',1.5
8,0,HW,CHKR,Chukar,Chukar Habitat,97c6f7c74f704f84b687eb0746cc07ec,VALUE=='crucial',1.5
9,0,HW,BISN,Bison,Bison Habitat,9bed043eb87641db88424d1080d03045,VALUE=='crucial',1.5


In [25]:
#Check for duplicate codes:
dfOpenDataLayers['LayerCode'].duplicated().any()

False

In [30]:
bProcess         = True  # toggle for entire cell... Set to false if you do not want to download and do not want to rejoin to projects
bReDownloadAll   = False # Set to True if you want to redownload data... otherwise set to False, this will download data and store in intermediate step... makes joining faster
bReJoinAll       = True  # Set to True if you want to rejoin the projects to the open data layers

gis = GIS(expiration=9999) # set expiration higher so it doesn't time out with the wetlands

if bProcess: 

    import time

    start_time = time.time()

    for index, row in dfOpenDataLayers.iterrows():

        # display all the variables used
        print (row['LayerCode'] + ' -  ' + row['OpenDataKey'] + ' - ' + str(row['SearchBuffer']) + ' -  ' + row['QueryFilter'])

        _searchBuffer = row['SearchBuffer']
        _layerCode = row['LayerCode']
        #try:
        _fc_download = os.path.join(intermediate_folder,'process.gdb\\' + _layerCode + '_orig_fltr')

        # check to see if replace all toggle is on OR if the feature class already exists in the gdb
        if bReDownloadAll or not arcpy.Exists(_fc_download) or row['Download']:

            # delete feature class if it exists
            delIfExists(_fc_download)

            # get data from web
            print ("downloading...")
            _item  = gis.content.get(row['OpenDataKey'])
            _flayer = _item.layers[0]
            _sdfLyr = pd.DataFrame.spatial.from_layer(_flayer)

            # filter data based on query
            print ("filtering...")
            if bool(row['QueryFilter']):
                _sdfLyr = _sdfLyr.query(row['QueryFilter'])

            # if sdf is not empty
            if not _sdfLyr.empty:
                print ("exporting intermediate layer...")
                _sdfLyr.spatial.to_featureclass(_fc_download)

        if bReJoinAll:

            # only join again if download has been created
            if arcpy.Exists(_fc_download):

                # Loop through the rows of the original dataframe
                for index, row in dfProjects.iterrows():

                    _fcPrjLayer = row['prjGroup'] + '_' + row['prjType']
                    _fcPrjIn  = os.path.join(intermediate_folder,'process.gdb\\_prj_' + _fcPrjLayer)

                    if row['prjType']=='lines':
                        _fcPrj = _fcPrjIn + '_segs'
                    elif row['prjType']=='points':
                        _fcPrj = _fcPrjIn

                    _fcPrj_joined = _fcPrj + '_' + _layerCode + '_joined'

                    # delete feature class if it exists
                    delIfExists(_fcPrj_joined)

                    # get closest distance
                    print (_fcPrjLayer + " joining...")
                    arcpy.analysis.SpatialJoin(_fcPrj, _fc_download, _fcPrj_joined, "JOIN_ONE_TO_ONE", "KEEP_COMMON", match_option="CLOSEST", search_radius=str(_searchBuffer) + " mile", distance_field_name="meters" + _layerCode)
            else:
                print('no features to join to')
                
        end_time = time.time()

        hours, rem = divmod(end_time - start_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"done... elapsed time: {int(hours)} hours, {int(minutes)} minutes, {int(seconds)} seconds") 
        #except:
        #    print ("ERROR: " + row['LayerCode'])

print ('ALL DONE!')

LOWN -  929fe39efa1d43ac8b151c2898ad3efe - 1.5 -  OWNER!='Private'
Roadway_points joining...
Roadway_lines joining...
Transit_points joining...
Transit_lines joining...
AT_points joining...
AT_lines joining...
done... elapsed time: 0 hours, 0 minutes, 8 seconds
AGFL -  875527c1e41e40d8b52340f18e89fb52 - 1.5 -  
Roadway_points joining...
Roadway_lines joining...
Transit_points joining...
Transit_lines joining...
AT_points joining...
AT_lines joining...
done... elapsed time: 0 hours, 0 minutes, 18 seconds
ELKS -  224dbcbdca174d3e8648a8b5693fd130 - 1.5 -  VALUE=='crucial'
Roadway_points joining...
Roadway_lines joining...
Transit_points joining...
Transit_lines joining...
AT_points joining...
AT_lines joining...
done... elapsed time: 0 hours, 0 minutes, 25 seconds
DEER -  a4ea235cef824862b560c693b4f7959b - 1.5 -  VALUE=='crucial'
Roadway_points joining...
Roadway_lines joining...
Transit_points joining...
Transit_lines joining...
AT_points joining...
AT_lines joining...
done... elapsed ti

In [54]:
# Loop through the rows of the original dataframe
for index, row in dfProjects.iterrows():

    _prjType = row['prjType']
    # project group and type
    _fcPrjLayer = row['prjGroup'] + '_' + _prjType
    print(_fcPrjLayer)

    _fcPrjIn  = os.path.join(intermediate_folder,'process.gdb\\_prj_' + _fcPrjLayer)

    if row['prjType']=='lines':
        _fcPrj = _fcPrjIn + '_segs'
    elif row['prjType']=='points':
        _fcPrj = _fcPrjIn

    dfD = pd.DataFrame()

    for index, row in dfOpenDataLayers.iterrows():
        #try:
        print(row['LayerCode'])
        _fc = _fcPrj + '_' + row['LayerCode'] + '_joined'
        if arcpy.Exists(_fc):
            #display(_fc)

            _sdf = pd.DataFrame.spatial.from_featureclass(_fc)
            _sdf.columns = _sdf.columns.str.upper()

            # add sequence for points, since they don't have segments
            if _prjType=='points':
                _sdf['ORIG_SEQ'] = 0

            df = _sdf[['GIS_ID',"METERS" + row['LayerCode'],'ORIG_SEQ']].copy()
            df['LayerCode'] = row['LayerCode']

            df.columns=(['GIS_ID','dist_meters','ORIG_SEQ','LayerCode'])

            #display(_sdf[['GIS_ID',"meters" + row['LayerCode']]])

            dfD = pd.concat([dfD,df])

    dfD['dist_feet'] = dfD['dist_meters'] / 3.28084

    dfD = dfD.drop(columns=('dist_meters'))

    # DON'T USE NONE OR NEW
    dfDistance = dfD[['GIS_ID','dist_feet','ORIG_SEQ','LayerCode']].copy()
    dfDistance.columns =['g','ft','s','l']
    dfDistance['id'] = dfDistance['g'].astype(str) + '_' + dfDistance['s'].astype(str)
    dfDistance = dfDistance[['l','id','ft']]
    dfDistance = dfDistance.groupby(['l','id'],as_index=False).agg(ft=('ft','mean'))
    dfDistance['ft'] = dfDistance['ft'].astype(int)
    #dfDistance

    dfDistance_pivot = dfD.pivot_table(index=['GIS_ID','ORIG_SEQ'],columns='LayerCode',aggfunc='mean')
    dfDistance_pivot.fillna(-1,inplace=True)
    dfDistance_pivot = dfDistance_pivot.astype(int)
    dfDistance_pivot.columns = dfDistance_pivot.columns.droplevel(0)
    dfDistance_pivot.reset_index(inplace=True)
    #dfDistance_pivot

    sdfPrj = pd.DataFrame.spatial.from_featureclass(_fcPrj)
    sdfPrj.columns = sdfPrj.columns.str.upper()
    if _prjType=="points":
        sdfPrj['ORIG_SEQ'] = 0
    sdfPrj = sdfPrj.groupby(['GIS_ID','ORIG_SEQ'],as_index=False).agg(SHAPE=('SHAPE','first'))

    #display(sdfRtpLinesSegs)

    sdfPrjWithDist = pd.DataFrame.merge(sdfPrj,dfDistance_pivot,on=('GIS_ID','ORIG_SEQ'),how='left')

    # iterate through the list of columns and add them to the DataFrame
    for col in dfOpenDataLayers['LayerCode'].unique():
        if col not in sdfPrjWithDist.columns:
            sdfPrjWithDist[col] = None
            
    sdfPrjWithDist.fillna(-1,inplace=True)

    #display(sdfPrjWithDist)

    # export segs
    bExportSegs = True

    if bExportSegs:
        sdfPrjWithDist.spatial.to_featureclass(os.path.join(results_folder,"resiliency_" +_fcPrjLayer + "_segs.shp"), sanitize_columns=False)
        print("SEGMENTS EXPORTED")
        sdfPrjIn = pd.DataFrame.spatial.from_featureclass(_fcPrjIn)[['GIS_ID','SHAPE']]
        sdfPrjIn.spatial.to_featureclass(os.path.join(results_folder,"resiliency_projects_lines.shp"))
        print("PROJECTS EXPORTED")
    else:
        print("SEGMENTS NOT EXPORTED")
        print("PROJECTS NOT EXPORTED")

    # get all gisids
    sdfPrjIn[['GIS_ID']].drop_duplicates().rename(columns={'GIS_ID':'g'}).to_json(os.path.join(folderWebAppDataFolder, _fcPrjLayer + '_gids.json'),index=False,orient="table")

    # get only gisid and seq of 
    dfDistance_pivot[['GIS_ID','ORIG_SEQ']].drop_duplicates().rename(columns={'GIS_ID':'g','ORIG_SEQ':'s'}).to_json(os.path.join(folderWebAppDataFolder, _fcPrjLayer + '_segs.json'),index=False,orient="table")

    # REDO

    # FILE BY BUFFER - GIS_ID - CATEGORY
    # ROW BY ORIG_SEQ
    # RECORDS: 

    lstBuffers = [100,200,300,400,500,600]

    dfBuffers = dfDistance.copy()

    for b in lstBuffers:
        print(b)
        dfBuffers.loc[(dfBuffers['ft']<b),'b' + str(b)] = 1
        dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].fillna(0)
        dfBuffers['b' + str(b)] = dfBuffers['b' + str(b)].astype(int)


    #display(dfBuffers)
    dfBuffers.drop(columns=('ft'),inplace=True)

    #display(dfBuffers)
    # export one dist file per gis id

    #for index, row in dfDistance[['l']].drop_duplicates().iterrows():
    #    print(row['l'])
#    _dfExport = dfDistance[dfDistance['l']==row['l']][['id','d']].set_index('id').to_json(os.path.join(folderWebAppDataFolder + '\\dist' , row['l'] + '.json'),orient='index')

    dfBuffersWithCat = pd.DataFrame.merge(dfBuffers, dfOpenDataLayers, left_on='l', right_on='LayerCode')
    #display(dfBuffersWithCat)

    for b in lstBuffers:
        print(b)
        for c in dfCategories['CategoryCode'].unique():
            print(c)
            _lyrs = (dfOpenDataLayers[dfOpenDataLayers['CategoryCode']==c]['LayerCode'].unique())
            _df = dfBuffersWithCat[(dfBuffersWithCat['CategoryCode']==c)][['id','l','CategoryCode','b' + str(b)]]
            _df = _df[_df['b' + str(b)]>0]
            _df.loc[(_df['b' + str(b)]==1),'value'] = _df['l']
            #display(_df)

            _dfp = _df.pivot(index='id', columns='l', values='value')
            _dfp.reset_index(inplace=True)
            _dfp.fillna('',inplace=True)
            _dfp[c] = _dfp[_df['l'].unique()].apply(lambda x: ",".join(filter(None, x)), axis =1)
            _dfp = _dfp[['id',c]]
            _dfp.set_index('id',inplace=True)
            #display(_dfp)

            _dfp.to_json(os.path.join(folderWebAppDataFolder + '\\within_buffers' , _fcPrjLayer + '_b' + str(b) + '_' + c + '.json'),orient='index')

Roadway_points
LOWN
AGFL
ELKS
DEER
MOOS
BEAR
GOAT
DGRS
CHKR
BISN
CSTG
CAQU
CABS
BTPG
BFWT
BFNT
HZWO
SWLN
FUDS
MMRP
NPRL
SFST
ICBZ
TXRI
VCUP
UHDS
EQFA
LCPK
PAPR
FLDP
QUFL
STSL
RIPA
WETL
SEGMENTS EXPORTED
PROJECTS EXPORTED
100
200
300
400
500
600
100
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
200
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
300
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
400
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
500
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
600
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
Roadway_lines
LOWN
AGFL
ELKS
DEER
MOOS
BEAR
GOAT
DGRS
CHKR
BISN
CSTG
CAQU
CABS
BTPG
BFWT
BFNT
HZWO
SWLN
FUDS
MMRP
NPRL
SFST
ICBZ
TXRI
VCUP
UHDS
EQFA
LCPK
PAPR
FLDP
QUFL
STSL
RIPA
WETL
SEGMENTS EXPORTED
PROJECTS EXPORTED
100
200
300
400
500
600
100
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
200
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
300
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
400
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
500
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
600
AF
CH
EJ
FP
GH
HW
HM
HY
OS
SS
Transit_points
LOWN
AGFL
ELKS
DEER
MOOS
BEAR
GOAT
DGRS
CHKR
BISN
CSTG
CAQU
CABS
BTPG
BFWT
BFNT
HZWO
SWL

In [55]:
# export jsons for categories and layers
dfCategories    .to_json(os.path.join(folderWebAppDataFolder,'cats.json'),orient="table")
dfOpenDataLayers.to_json(os.path.join(folderWebAppDataFolder,'lyrs.json'),orient="table")